In [4]:
import os
from pymskt.mesh import Mesh, BoneMesh, CartilageMesh
    
import os
from pymskt.mesh import Mesh
import SimpleITK as sitk
import numpy as np

from pymskt.image import read_nrrd
from pymskt.mesh.meshTransform import SitkVtkTransformer
from pymskt.mesh.meshTools import ProbeVtkImageDataAlongLine, get_cartilage_properties_at_points
from pymskt.mesh.meshTools import get_surface_normals, n2l, l2n
from pymskt.mesh.utils import is_hit, get_intersect, get_surface_normals, get_obb_surface

In [ ]:
bone_mesh_path = '/dataNAS/people/anoopai/DESS_ACL_study/results/BScore_and_FC_metrics/mean_shape_recon/BC_model/interpolation/ctrl_Visit-1_Bscore0.16_bone.vtk'
cart_mesh_path = '/dataNAS/people/anoopai/DESS_ACL_study/results/BScore_and_FC_metrics/mean_shape_recon/BC_model/interpolation/ctrl_Visit-1_Bscore0.16_cart.vtk'

bone_mesh= Mesh(bone_mesh_path)
cart_mesh = Mesh(cart_mesh_path)

# IMPORTANT: The meshes should be "unnormailised" o patient size. Rough factor for scale-up is 45
bone_mesh.point_coords = bone_mesh.point_coords * 45
cart_mesh.point_coords = cart_mesh.point_coords * 45

# setup the probe that we are using to get data from the T2 file 
line_resolution = 500   # number of points along the line that the T2 data is sampled at
no_thickness_filler = 0.0              # if no data is found, what value to fill the data with
ray_cast_length= 20          # how far to extend the ray from the surface (using negative to go inwards/towards the other side)
percent_ray_length_opposite_direction = 0.025  # extend the other way a % of the line to make sure get both edges. 1.0 = 100%|
n_intersections = 2  # how many intersections to expect. If 2, then we expect to find two points on the ray that intersect the cartilage surface.

thicknesses = np.zeros(bone_mesh.GetNumberOfPoints())

node_data = get_cartilage_properties_at_points(
    bone_mesh,
    cart_mesh,
    t2_vtk_image=None,
    seg_vtk_image=None,
    ray_cast_length=ray_cast_length,
    percent_ray_length_opposite_direction=percent_ray_length_opposite_direction,
)
thicknesses += node_data

# Assign the thickness scalars to the bone mesh surface.
bone_mesh.point_data["thickness (mm)"] = thicknesses
bone_mesh.save_mesh(bone_mesh_path)

INTERSECTION IS:  2


In [12]:
np.unique(node_data)

array([0.00000000e+00, 1.48324575e-06, 4.03695758e-05, ...,
       2.80267087e-01, 3.03454380e-01, 3.35764091e-01])